# 모듈 import

In [1]:
import pickle
import numpy as np
import pandas as pd


import easydict


#시각화 모듈
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
import seaborn as sns




import scipy as sp
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor




from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import roc_curve, accuracy_score, confusion_matrix, recall_score, precision_score,f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


import pandas as pd
from datetime import datetime, date

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, accuracy_score, confusion_matrix, recall_score, precision_score,f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import statsmodels.api as sm

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from tensorflow.keras import activations
import itertools
import time
import random as rn
RANDOM_SEED = 5
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
tf.random.set_seed(
    RANDOM_SEED
)

from sklearn.neighbors import KNeighborsRegressor
from math import sqrt


import matplotlib

import warnings
warnings.filterwarnings('ignore')

# 한글폰트 사용시 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [9]:
# 그래프에 마이너스 표시가 되도록 변경
matplotlib.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_rows', 200)



"""pickle 불러오는 함수"""
def load_pickle(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

"""리드 타임이 하루 이상 지났으면 제거"""
def lead_none_delete(alrm_data):
    alrm_data = alrm_data[alrm_data['ALRM_LEADTIME']<86400].reset_index(drop=True)
    return alrm_data

"""feature enginnering을 위한 병렬처리 함수"""
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores) # core의 개수만큼 df를 나눔
    pool = Pool(n_cores) # pool을 core개수만큼 생성
    df = pd.concat(pool.map(func, df_split)) # 나누어진 df를 func을 적용해서 수행 및 concat
    pool.close()
    pool.join() # 모두 완료될 때까지 대기
    return df

"""datetime 형식으로 컬럼 변경"""
def transform_datetype(df, columns):
    df[columns] = df[columns].astype('str')
    df[columns] = pd.to_datetime(df[columns])
    return df

"""센서별 알람 구간"""
def alrm_range(df):
    ## 알람이 울린 구간을 tuple로 저장
    range_list = list(df.apply(lambda x : tuple((x['ALRM_TMSTP'], x['END_TMSTP'])), axis=1))
    return range_list

# 데이터 EDA

In [4]:
trnd_data = pd.read_csv("TH_CM_FMCS_TRND_DATA.csv")
A_5차 = load_pickle('A_5차.pickle')
B_5차 = load_pickle('B_5차.pickle')
ETC_5차 = load_pickle('ETC_5차.pickle')
설비번호맵핑 = pd.read_excel("설비번호맵핑.xlsx")
설비번호맵핑 = 설비번호맵핑[['EQP_NO','ORDER']]
grade_B_alrm = pd.read_excel('B등급 알람센서.xlsx', sheet_name='Sheet2')

In [5]:
trnd_data

,EQP_NO,PARAM_CODE,TRND_DATE,TAG_COMPRS_VAL
0,4067851,OPLAST,22/04/19 15:40:00,0:0.8569;3:0.9009;6:1.1026;9:1.2948;12:1.4507;...
1,4068165,PVLAST,22/04/19 15:40:00,0:1.0000;
2,4096196,OPLAST,22/04/19 15:40:00,0:41.6477;3:41.6486;6:41.6459;9:41.6486;12:41....
3,4067237,PVLAST,22/04/19 15:40:00,0:0.2604;9:0.2894;12:0.2604;18:0.2894;21:0.260...
4,4067456,PVLAST,22/04/19 15:40:00,0:49.6007;15:49.6354;18:49.6007;21:49.6354;24:...
...,...,...,...,...
967478,4067240,PVLAST,22/04/13 00:00:00,0:0.0579;
967479,4067853,OPLAST,22/04/13 00:00:00,0:52.3011;36:52.2611;39:52.3010;42:52.3009;45:...
967480,4070068,PVLAST,22/04/13 00:00:00,0:18.4486;3:18.5354;6:18.6560;9:18.7006;12:18....
967481,4086472,PVLAST,22/04/13 00:00:00,0:41.7499;6:41.7462;12:41.7499;15:41.7462;18:4...
